In [1]:
import cv2
import numpy as np
from numpy import cos, sin, deg2rad, roll, logical_and, where, inner, exp, rad2deg, arctan2, trace, dot, convolve, sqrt, subtract, log, floor, stack, delete, concatenate, max
from numpy.linalg import det, lstsq, norm
from einops import rearrange
from sift import getKernelSizes, getGaussianImages, getDoG, getScaleSpaceExtrema, sift, gaussianBlurring

In [2]:
# Load the images
image1 = cv2.imread('images/test1.jpeg', cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread('images/test2.jpeg', cv2.IMREAD_GRAYSCALE)

# reduce image size
xscale = 0.25
yscale = 0.25
image1_small = cv2.resize(image1, (0,0), fx = xscale, fy = yscale)
image2_small = cv2.resize(image2, (0,0), fx = xscale, fy = yscale)

# # Initialize SIFT detector
# sift = cv2.SIFT_create()

# # Detect keypoints and compute descriptors
# keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
# keypoints2, descriptors2 = sift.detectAndCompute(image2, None)

keypoints1, descriptors1 = sift(image1_small)
keypoints2, descriptors2 = sift(image2_small)



[1.         0.76642094 0.96562987 1.2166174  1.53284187 1.93125974]


/home/ansonhon/anson/projects/my_computer_vision/sift.py:105: RuntimeWarning: overflow encountered in scalar subtract
  dx = 0.5 * (pixel_cube[1,1,2] - pixel_cube[1,1,0])
/home/ansonhon/anson/projects/my_computer_vision/sift.py:117: RuntimeWarning: overflow encountered in scalar subtract
  dxs = 0.25 * (pixel_cube[2,1,2] - pixel_cube[2,1,0] - pixel_cube[0,1,2] +pixel_cube[0,1,0])
/home/ansonhon/anson/projects/my_computer_vision/sift.py:117: RuntimeWarning: overflow encountered in scalar add
  dxs = 0.25 * (pixel_cube[2,1,2] - pixel_cube[2,1,0] - pixel_cube[0,1,2] +pixel_cube[0,1,0])
/home/ansonhon/anson/projects/my_computer_vision/sift.py:118: RuntimeWarning: overflow encountered in scalar subtract
  dys = 0.25 * (pixel_cube[2,2,1] - pixel_cube[2,0,1] - pixel_cube[0,2,1] +pixel_cube[0,0,1])
/home/ansonhon/anson/projects/my_computer_vision/sift.py:118: RuntimeWarning: overflow encountered in scalar add
  dys = 0.25 * (pixel_cube[2,2,1] - pixel_cube[2,0,1] - pixel_cube[0,2,1] +pixel_cube

In [ ]:

# Initialize the Brute-Force matcher with L2 norm
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

# Match descriptors using KNN algorithm (k=2)
matches = bf.knnMatch(descriptors1, descriptors2, k=2)

# Apply Lowe's ratio test
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

# Draw matches
matched_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
# matched_image_small = cv2.resize(matched_image, (0,0), fx = 0.5, fy = 0.5)


# Display the matched image
cv2.imshow('Matches', matched_image)
cv2.waitKey(0)
cv2.destroyAllWindows()